# Data Preparation

**Script Objective:** to prepare Thai government budgetary excel file for policy analysis.

**Version 1.0** updated by Jay Sirabhop

## 1. Prelim step

Import library

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Deploy function

In [4]:
#For tracking data
def track_status(df, col_list):
  print('Row:', "{:,}".format(len(df.index)), 'and', 'Columns:', "{:,}".format(len(df.columns)))
  for i in range(len(col_list)):
    print(col_list[i]+':', "{:,}".format(df[col_list[i]].sum()))

In [4]:
def extract_from_specific_word(df, col_to_extract, word, range_):
  to_extract = df[col_to_extract].values
  extracted = []
  for i in range(len(to_extract)):
    km_index = str(to_extract[i]).find(word)
    if km_index != -1:
      km = to_extract[i][km_index: km_index + range_]
      km = ''.join(re.findall("[0-9]", km))
      extracted.append(str(km))
    else:
      extracted.append(np.NaN)
  return extracted

In [5]:
def find_start_end(df, col_n_start, col_n_end):
  start = []
  end = []
  years = df.columns[col_n_start:col_n_end]
  r_years = list(reversed(years))
  nrow = len(df.index)
  ncol = len(years)
  for row_number in range(nrow):
    for col_number in range(ncol):
      if not np.isnan(df.loc[row_number, years[col_number]]):
        s = years[col_number]
        break
    for col_number in range(ncol):
      if not np.isnan(df.loc[row_number, r_years[col_number]]):
        e = r_years[col_number]
        break
    print(row_number, s, e)
    start.append(s)
    end.append(e)
  return start, end

Import data

In [21]:
df = pd.read_excel('Data/งบประมาณประเทศไทย 2566 (ฉบับร่างพ.ร.บ.) - PDF to CSV.xlsx', sheet_name = 'RELEASE_22-05-27')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52031 entries, 0 to 52030
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REF_DOC           52031 non-null  object 
 1   REF_PAGE_NO       52031 non-null  float64
 2   MINISTRY          52031 non-null  object 
 3   BUDGETARY_UNIT    52031 non-null  object 
 4   BUDGET_PLAN       52031 non-null  object 
 5   CROSS_FUNC?       52031 non-null  bool   
 6   PROJECT           25017 non-null  object 
 7   OUTPUT            24648 non-null  object 
 8   CATEGORY_LV1      52019 non-null  object 
 9   CATEGORY_LV2      49580 non-null  object 
 10  CATEGORY_LV3      33257 non-null  object 
 11  CATEGORY_LV4      22755 non-null  object 
 12  CATEGORY_LV5      457 non-null    object 
 13  CATEGORY_LV6      0 non-null      float64
 14  ITEM_DESCRIPTION  52029 non-null  object 
 15  FISCAL_YEAR       52031 non-null  float64
 16  OBLIGED?          52031 non-null  bool  

## 2. Clean Data

In [7]:
track_status(df, ['AMOUNT'])

Row: 52,031 and Columns: 18
AMOUNT: 4,171,104,770,628.0


### Get province

In [22]:
df.insert(18, 'PROVINCE', np.NaN)
#Import province
province = pd.read_excel('Data/25640531_sc002_.xlsx', usecols = ['ProvinceNameThai', 'RegionName'])
province.rename(columns = {'ProvinceNameThai': 'PROVINCE', 'RegionName': 'REGION'}, inplace = True)
#Type 1: full province
province_type1_list = province.PROVINCE.to_list()
#Type 2: in case the item_description doesn't contain จังหวัด
province_type2_list = []
for i in range(len(province_type1_list)):
  province_type2_list.append(province_type1_list[i].replace('จังหวัด', ''))
for i in range(len(province_type2_list)):
  df.loc[df.ITEM_DESCRIPTION.str.contains(province_type2_list[i], na = False), 
         'PROVINCE'] = province_type1_list[i]
#join for region
df = df.merge(right = province, on = 'PROVINCE', how = 'left')

In [27]:
df.loc[df['ITEM_DESCRIPTION'].str.contains('กาฟัสินธุ์', na = False), 'PROVINCE'] = 'จังหวัดกาฬสินธุ์'

In [28]:
track_status(df, ['AMOUNT'])

Row: 52,031 and Columns: 20
AMOUNT: 4,171,104,770,628.0


### Get unique ITEM_DESCRIPTION and insert START_YEAR and END_YEAR

Pivot data

In [15]:
df_distinct = df.pivot_table(values = 'AMOUNT', index = 'ITEM_DESCRIPTION', columns = 'FISCAL_YEAR', aggfunc = sum)
df_distinct['AMOUNT'] = df_distinct.sum(axis = 1)
df_distinct.reset_index(inplace = True)

In [16]:
track_status(df_distinct, ['AMOUNT'])

Row: 22,613 and Columns: 102
AMOUNT: 4,171,088,539,728.0


Join 2 table and discard duplicate row

In [17]:
col_to_join = ['REF_DOC', 'REF_PAGE_NO', 'MINISTRY', 'BUDGETARY_UNIT', 'BUDGET_PLAN',
                'CROSS_FUNC?', 'PROJECT', 'OUTPUT', 'CATEGORY_LV1', 'CATEGORY_LV2',
                'CATEGORY_LV3', 'CATEGORY_LV4', 'CATEGORY_LV5', 'CATEGORY_LV6',
                'ITEM_DESCRIPTION', 'OBLIGED?', 'PROVINCE']

In [18]:
df_merged = pd.merge(left = df_distinct, right = df[col_to_join], on = 'ITEM_DESCRIPTION', how = 'left', validate = 'one_to_many')
track_status(df_merged, ['AMOUNT'])

Row: 52,029 and Columns: 118
AMOUNT: 252,635,233,963,840.0


In [19]:
df_merged = df_merged.drop_duplicates(subset = ['ITEM_DESCRIPTION'], keep = 'first')
track_status(df_merged, ['AMOUNT'])

Row: 22,613 and Columns: 118
AMOUNT: 4,171,088,539,728.0


In [ ]:
df_merged.reset_index(drop = True, inplace = True)
start, end = find_start_end(df_merged, 1, 101)

In [21]:
df_merged['START'] = start
df_merged['END'] = end

In [22]:
track_status(df_merged, ['AMOUNT'])

Row: 22,613 and Columns: 120
AMOUNT: 4,171,088,539,728.0


In [23]:
df_merged['DURATION'] = df_merged['END'] - df_merged['START']
track_status(df_merged, ['AMOUNT'])

Row: 22,613 and Columns: 121
AMOUNT: 4,171,088,539,728.0


In [24]:
df_merged[['START', 'END', 'DURATION']] = df_merged[['START', 'END', 'DURATION']].astype(int)
track_status(df_merged, ['AMOUNT'])

Row: 22,613 and Columns: 121
AMOUNT: 4,171,088,539,728.0


### Extract main type

In [11]:
df_merged['MAIN'] = df_merged['ITEM_DESCRIPTION'].str.split(expand = True)[0]

In [12]:
track_status(df_merged, ['AMOUNT'])

Row: 22,613 and Columns: 122
AMOUNT: 4,171,088,539,728.0


### Get Region

In [19]:
df_merged = df_merged.merge(province, how = 'left', on = 'PROVINCE')

## 3. Export Data

In [20]:
df_merged.to_excel('Data/budget_df.xlsx', index = False, sheet_name = 'raw')